## Sequence data
ex) 문장 (앞 뒤 단어들이 문장을 이해하는데 영향을 준다)
- NN/CNN can't do this

<img src='rnn.png'>
출처: http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* weight값은 모든 A에 동일

<img src='rnn1.png'>
- 첫 ht을 계산할 때 ht-1은 0으로 둔다

#### Character-level language model
Ex) 'hello'가 있을 때 [h,e,l,l,o]로 나눠서 각각을 x값으로 두고, 그 x값 다음에 무슨 character가 올 지 예측하는 것



# Lab12-1. RNN Basics

In [12]:
import tensorflow as tf
import pprint
import numpy as np

pp = pprint.PrettyPrinter()

In [9]:
#One cell RNN input_dim (4)  => output_dim (2)
tf.reset_default_graph() 

hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units= hidden_size)

x_data = np.array([[[1,0,0,0]]], dtype = np.float32)
outputs, _status = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)

config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval(session = sess))

array([[[0.09200777, 0.10546921]]], dtype=float32)


## Unfolding to n sequences
- hidden_size = 2
- sequence_length = 5

In [10]:
#One cell RNN input_dim(4) -> output_dim (2), sequence: 5
tf.reset_default_graph() 
#one hot encoding
h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

hidden_size = 2
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size) #cell 생성
x_data = np.array([[h, e, l, l,o]], dtype = np.float32)
print(x_data.shape) #dim = 4, len = 5
pp.pprint(x_data)

outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32) #y값들

config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval(session = sess))

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[ 0.02213394,  0.10033629],
        [-0.05361217,  0.15283416],
        [-0.10702882,  0.14590324],
        [-0.1495565 ,  0.09748948],
        [-0.1895167 ,  0.17559034]]], dtype=float32)


### Batching input
- 여러개의 입력 데이터

In [11]:
tf.reset_default_graph()

x_data = np.array([[h, e, l, l, o],
                  [e, o, l, l, l],
                  [l,l,e,e,l]], dtype = np.float32)

pp.pprint(x_data)

cell = tf.contrib.rnn.BasicLSTMCell(num_units = 2, state_is_tuple = True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)

config = tf.ConfigProto(device_count={'GPU': 0})
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval(session = sess))

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.0935028 ,  0.01764144],
        [ 0.05650486, -0.03605662],
        [ 0.02010914,  0.05039952],
        [-0.00023759,  0.126472  ],
        [-0.06484948,  0.1882711 ]],

       [[ 0.02047359, -0.03717672],
        [-0.06881153,  0.06259921],
        [-0.03896397,  0.15046191],
        [-0.02340028,  0.20319901],
        [-0.01204985,  0.23564003]],

       [[-0.0142216 ,  0.09695118],
        [-0.0144067 ,  0.16372035],
        [ 0.02792709,  0.0541846 ],
        [ 0.04040593,  0.00236255],
        [ 0.0121753 ,  0.09131035]]], dtype=float32)


# Lab12-2. Teach RNN 'hihello'
- 같은 h에도 다른 값이 나와야 하기 때문에 기존의 forward로는 하기 어려움

In [23]:
tf.reset_default_graph()
#RNN parameters
hidden_size = 5
input_dim = 5
batch_size = 1 #one sentence
sequence_length = 6  

#Data creation
idx2char = ['h', 'i', 'e','l', 'o']
x_data = [[0,1,0,2,3,3]] #hihell
x_one_hot = [[[1,0,0,0,0],
              [0,1,0,0,0],
              [1,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,1,0],
              [0,0,0,1,0]]]

y_data = [[1,0,2,3,3,4]] #ihello
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim]) #X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])

In [24]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state = initial_state, dtype = tf.float32)

### Cost: sequence_loss
output이 얼마나 좋은가를 loss를 통해 측정

In [15]:
# [ batch_size, sequence_length]
sess.close()

y_data = tf.constant([[1,1,1]])

prediction1 = tf.constant([[[0.3, 0.7],[0.3, 0.7], [0.3, 0.7]]], dtype = tf.float32)
prediction2 = tf.constant([[[0.1,0.9],[0.1,0.9],[0.1,0.9]]], dtype = tf.float32)

weights = tf.constant([[1,1,1]], dtype = tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)

sess = tf.Session()
sess.run(tf.global_variables_initializer())


print('Loss1: ', sequence_loss1.eval(session = sess), 
     'Loss2: ', sequence_loss2.eval(session = sess))  #당연히 더 작은 loss!



Loss1:  0.5130153 Loss2:  0.37110072


In [26]:
weights = tf.ones([batch_size, sequence_length])

#원래는 logit에 outputs 바로 쓰면 안 된다. 
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights = weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [28]:
prediction = tf.argmax(outputs, axis = 2)

config = tf.ConfigProto(device_count = {'GPU':0})
with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict = {X:x_one_hot, Y:y_data})
        result = sess.run(prediction, feed_dict = {X:x_one_hot})
        print(i, 'loss: ', l, 'prediction: ', result, 'true Y: ', y_data)
        
        #print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print('\tPredictioin str: ', ''.join(result_str))

0 loss:  1.5953269 prediction:  [[3 0 0 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  lhhllo
1 loss:  1.5202985 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  llllll
2 loss:  1.4512643 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  llllll
3 loss:  1.3753545 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  llllll
4 loss:  1.3031923 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  llllll
5 loss:  1.241281 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  llllll
6 loss:  1.1919924 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  llllll
7 loss:  1.1468854 prediction:  [[3 0 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  lhllll
8 loss:  1.1070879 prediction:  [[3 0 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  lhllll
9 loss:  1.0750444 prediction:  [[3 0 3 3 3 3]] true Y:  [[1, 0, 

98 loss:  0.65937525 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
99 loss:  0.6591353 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
100 loss:  0.6589019 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
101 loss:  0.65867543 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
102 loss:  0.65845495 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
103 loss:  0.6582396 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
104 loss:  0.6580288 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
105 loss:  0.6578228 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
106 loss:  0.65762186 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
107 loss:  0.6574255 prediction:  [[1 0 2 3 

196 loss:  0.6485073 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
197 loss:  0.6484523 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
198 loss:  0.6483977 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
199 loss:  0.64834374 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
200 loss:  0.6482902 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
201 loss:  0.6482372 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
202 loss:  0.648185 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
203 loss:  0.6481334 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
204 loss:  0.648083 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
205 loss:  0.6480336 prediction:  [[1 0 2 3 3 4

286 loss:  0.64491457 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
287 loss:  0.6448856 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
288 loss:  0.6448569 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
289 loss:  0.6448285 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
290 loss:  0.64480084 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
291 loss:  0.6447735 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
292 loss:  0.6447474 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
293 loss:  0.6447215 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
294 loss:  0.6446985 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
295 loss:  0.6446751 prediction:  [[1 0 2 3 

378 loss:  0.64275634 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
379 loss:  0.6427406 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
380 loss:  0.64272475 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
381 loss:  0.6427121 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
382 loss:  0.6426981 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
383 loss:  0.64269006 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
384 loss:  0.6426758 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
385 loss:  0.6426708 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
386 loss:  0.64264965 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
387 loss:  0.6426378 prediction:  [[1 0 2 

...

1591 loss:  0.63508755 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1592 loss:  0.6350849 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1593 loss:  0.63508224 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1594 loss:  0.6350796 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1595 loss:  0.63507694 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1596 loss:  0.6350743 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1597 loss:  0.6350717 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1598 loss:  0.635069 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1599 loss:  0.6350664 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1600 loss:  0.63506377 prediction: 

1687 loss:  0.63484186 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1688 loss:  0.63483936 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1689 loss:  0.6348369 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1690 loss:  0.63483447 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1691 loss:  0.6348319 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1692 loss:  0.6348295 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1693 loss:  0.6348271 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1694 loss:  0.63482463 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1695 loss:  0.63482213 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1696 loss:  0.6348197 prediction

1783 loss:  0.6346126 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1784 loss:  0.63461035 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1785 loss:  0.634608 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1786 loss:  0.6346057 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1787 loss:  0.6346034 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1788 loss:  0.6346011 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1789 loss:  0.63459885 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1790 loss:  0.63459647 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1791 loss:  0.63459426 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1792 loss:  0.634592 prediction:  

1880 loss:  0.6343962 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1881 loss:  0.63439405 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1882 loss:  0.63439196 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1883 loss:  0.63438976 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1884 loss:  0.6343876 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1885 loss:  0.6343854 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1886 loss:  0.6343833 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1887 loss:  0.6343811 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1888 loss:  0.63437897 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1889 loss:  0.6343768 prediction:

1976 loss:  0.6341958 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1977 loss:  0.6341938 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1978 loss:  0.63419175 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1979 loss:  0.6341897 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1980 loss:  0.6341877 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1981 loss:  0.6341857 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1982 loss:  0.6341837 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1983 loss:  0.6341817 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1984 loss:  0.63417965 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
	Predictioin str:  ihello
1985 loss:  0.6341777 prediction:  

# Lab12-3: RNN with long sequences

### Better data creation

In [35]:
tf.reset_default_graph()

sample = 'if you want you'
idx2char = list(set(sample))
char2idx = {c:i for i, c in enumerate(idx2char)}

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

In [36]:
#hyper parameters
sequence_length = len(sample) - 1
num_classes = len(idx2char)
dic_size = len(char2idx)
rnn_hidden_size = len(char2idx)
batch_size = 1

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

X_one_hot = tf.one_hot(X, num_classes)

### LSTM and Loss

In [38]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units = rnn_hidden_size, state_is_tuple = True)
initial_state = cell.zero_state(batch_size, tf.float32) 
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, initial_state = initial_state, dtype = tf.float32)

weights = tf.ones([batch_size,sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y,
                                                weights = weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(loss)

prediction = tf.argmax(outputs, axis= 2)

In [40]:
with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(3000):
        l, _ = sess.run([loss, train], feed_dict = {X:x_data, Y:y_data})
        result = sess.run(prediction, feed_dict = {X:x_data})
        
        #print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, 'loss: ', l, 'Prediction: ', ''.join(result_str))

0 loss:  2.3233848 Prediction:  nwwaa   a tttt
1 loss:  2.3200145 Prediction:  nwwaa   a tttt
2 loss:  2.3167 Prediction:  nwwaa   a tttt
3 loss:  2.3134432 Prediction:  nwwta   a tttt
4 loss:  2.3102443 Prediction:  nwwta   a ttt 
5 loss:  2.307105 Prediction:  nwwta     t t 
6 loss:  2.3040261 Prediction:  nwnt        t 
7 loss:  2.3010075 Prediction:  nwnt        t 
8 loss:  2.2980494 Prediction:  nwnt        t 
9 loss:  2.2951524 Prediction:  n nt        t 
10 loss:  2.292316 Prediction:  n           t 
11 loss:  2.2895398 Prediction:  n           t 
12 loss:  2.2868233 Prediction:  n           t 
13 loss:  2.2841663 Prediction:  n             
14 loss:  2.2815676 Prediction:  n             
15 loss:  2.2790267 Prediction:  n             
16 loss:  2.2765424 Prediction:  n             
17 loss:  2.274114 Prediction:                
18 loss:  2.2717397 Prediction:                
19 loss:  2.2694192 Prediction:                
20 loss:  2.2671506 Prediction:                
21 loss:

219 loss:  2.105182 Prediction:                
220 loss:  2.1046848 Prediction:                
221 loss:  2.1041882 Prediction:                
222 loss:  2.1036918 Prediction:                
223 loss:  2.1031957 Prediction:                
224 loss:  2.1027 Prediction:                
225 loss:  2.102204 Prediction:                
226 loss:  2.1017087 Prediction:                
227 loss:  2.1012132 Prediction:                
228 loss:  2.1007183 Prediction:                
229 loss:  2.1002233 Prediction:                
230 loss:  2.0997286 Prediction:                
231 loss:  2.0992343 Prediction:                
232 loss:  2.0987399 Prediction:                
233 loss:  2.0982454 Prediction:                
234 loss:  2.0977516 Prediction:                
235 loss:  2.0972576 Prediction:                
236 loss:  2.0967638 Prediction:                
237 loss:  2.0962703 Prediction:                
238 loss:  2.0957766 Prediction:                
239 loss:  2.0952833 Pred

430 loss:  1.9958813 Prediction:                
431 loss:  1.9953057 Prediction:                
432 loss:  1.9947293 Prediction:                
433 loss:  1.9941518 Prediction:                
434 loss:  1.9935738 Prediction:                
435 loss:  1.9929949 Prediction:                
436 loss:  1.992415 Prediction:                
437 loss:  1.9918344 Prediction:                
438 loss:  1.991253 Prediction:                
439 loss:  1.9906704 Prediction:                
440 loss:  1.9900872 Prediction:                
441 loss:  1.9895031 Prediction:                
442 loss:  1.9889184 Prediction:                
443 loss:  1.9883323 Prediction:                
444 loss:  1.9877459 Prediction:                
445 loss:  1.9871582 Prediction:                
446 loss:  1.9865696 Prediction:                
447 loss:  1.9859805 Prediction:                
448 loss:  1.9853903 Prediction:                
449 loss:  1.9847991 Prediction:                
450 loss:  1.9842073 P

655 loss:  1.8380033 Prediction:    you       ou
656 loss:  1.8371363 Prediction:    you       ou
657 loss:  1.8362676 Prediction:    you       ou
658 loss:  1.8353971 Prediction:    you       ou
659 loss:  1.8345251 Prediction:    you       ou
660 loss:  1.8336511 Prediction:    you       ou
661 loss:  1.8327756 Prediction:    you       ou
662 loss:  1.8318983 Prediction:    you       ou
663 loss:  1.8310192 Prediction:    you       ou
664 loss:  1.8301386 Prediction:    you       ou
665 loss:  1.8292562 Prediction:    you       ou
666 loss:  1.8283722 Prediction:    you       ou
667 loss:  1.8274863 Prediction:    you       ou
668 loss:  1.8265989 Prediction:    you       ou
669 loss:  1.8257097 Prediction:    you       ou
670 loss:  1.8248188 Prediction:    you       ou
671 loss:  1.8239263 Prediction:    you       ou
672 loss:  1.8230321 Prediction:    you       ou
673 loss:  1.8221363 Prediction:    you       ou
674 loss:  1.8212388 Prediction:    you       ou
675 loss:  1.8203398

866 loss:  1.6428472 Prediction:    you      you
867 loss:  1.6420277 Prediction:    you      you
868 loss:  1.6412107 Prediction:    you      you
869 loss:  1.6403955 Prediction:    you      you
870 loss:  1.6395825 Prediction:    you      you
871 loss:  1.6387717 Prediction:    you      you
872 loss:  1.6379628 Prediction:    you      you
873 loss:  1.6371561 Prediction:    you      you
874 loss:  1.6363515 Prediction:    you      you
875 loss:  1.6355491 Prediction:    you      you
876 loss:  1.6347487 Prediction:    you      you
877 loss:  1.6339505 Prediction:    you      you
878 loss:  1.6331543 Prediction:    you w    you
879 loss:  1.6323602 Prediction:    you w    you
880 loss:  1.6315682 Prediction:    you w    you
881 loss:  1.6307782 Prediction:    you w    you
882 loss:  1.6299905 Prediction:    you w    you
883 loss:  1.6292046 Prediction:    you w    you
884 loss:  1.6284211 Prediction:    you w    you
885 loss:  1.6276395 Prediction:    you w    you
886 loss:  1.6268601

...

1991 loss:  1.2810551 Prediction:  f you want you
1992 loss:  1.2809386 Prediction:  f you want you
1993 loss:  1.2808222 Prediction:  f you want you
1994 loss:  1.2807058 Prediction:  f you want you
1995 loss:  1.2805897 Prediction:  f you want you
1996 loss:  1.2804736 Prediction:  f you want you
1997 loss:  1.2803577 Prediction:  f you want you
1998 loss:  1.2802422 Prediction:  f you want you
1999 loss:  1.2801265 Prediction:  f you want you
2000 loss:  1.280011 Prediction:  f you want you
2001 loss:  1.2798958 Prediction:  f you want you
2002 loss:  1.2797809 Prediction:  f you want you
2003 loss:  1.2796658 Prediction:  f you want you
2004 loss:  1.279551 Prediction:  f you want you
2005 loss:  1.2794365 Prediction:  f you want you
2006 loss:  1.2793219 Prediction:  f you want you
2007 loss:  1.2792076 Prediction:  f you want you
2008 loss:  1.2790931 Prediction:  f you want you
2009 loss:  1.2789792 Prediction:  f you want you
2010 loss:  1.2788653 Prediction:  f you want you
20

2217 loss:  1.2580466 Prediction:  f you want you
2218 loss:  1.2579578 Prediction:  f you want you
2219 loss:  1.2578692 Prediction:  f you want you
2220 loss:  1.2577807 Prediction:  f you want you
2221 loss:  1.2576921 Prediction:  f you want you
2222 loss:  1.2576036 Prediction:  f you want you
2223 loss:  1.2575154 Prediction:  f you want you
2224 loss:  1.2574272 Prediction:  f you want you
2225 loss:  1.2573392 Prediction:  f you want you
2226 loss:  1.2572513 Prediction:  f you want you
2227 loss:  1.2571633 Prediction:  f you want you
2228 loss:  1.2570755 Prediction:  f you want you
2229 loss:  1.2569879 Prediction:  f you want you
2230 loss:  1.2569003 Prediction:  f you want you
2231 loss:  1.2568128 Prediction:  f you want you
2232 loss:  1.2567253 Prediction:  f you want you
2233 loss:  1.2566382 Prediction:  f you want you
2234 loss:  1.256551 Prediction:  f you want you
2235 loss:  1.2564638 Prediction:  f you want you
2236 loss:  1.2563769 Prediction:  f you want you
2

2439 loss:  1.2405283 Prediction:  f you want you
2440 loss:  1.2404581 Prediction:  f you want you
2441 loss:  1.2403882 Prediction:  f you want you
2442 loss:  1.2403183 Prediction:  f you want you
2443 loss:  1.2402483 Prediction:  f you want you
2444 loss:  1.2401786 Prediction:  f you want you
2445 loss:  1.240109 Prediction:  f you want you
2446 loss:  1.2400393 Prediction:  f you want you
2447 loss:  1.2399698 Prediction:  f you want you
2448 loss:  1.2399004 Prediction:  f you want you
2449 loss:  1.239831 Prediction:  f you want you
2450 loss:  1.2397616 Prediction:  f you want you
2451 loss:  1.2396923 Prediction:  f you want you
2452 loss:  1.2396231 Prediction:  f you want you
2453 loss:  1.239554 Prediction:  f you want you
2454 loss:  1.239485 Prediction:  f you want you
2455 loss:  1.239416 Prediction:  f you want you
2456 loss:  1.239347 Prediction:  f you want you
2457 loss:  1.2392783 Prediction:  f you want you
2458 loss:  1.2392095 Prediction:  f you want you
2459 l

2661 loss:  1.2265518 Prediction:  f you want you
2662 loss:  1.2264953 Prediction:  f you want you
2663 loss:  1.2264389 Prediction:  f you want you
2664 loss:  1.2263825 Prediction:  f you want you
2665 loss:  1.2263262 Prediction:  f you want you
2666 loss:  1.22627 Prediction:  f you want you
2667 loss:  1.2262135 Prediction:  f you want you
2668 loss:  1.2261574 Prediction:  f you want you
2669 loss:  1.2261013 Prediction:  f you want you
2670 loss:  1.2260451 Prediction:  f you want you
2671 loss:  1.2259891 Prediction:  f you want you
2672 loss:  1.225933 Prediction:  f you want you
2673 loss:  1.2258773 Prediction:  f you want you
2674 loss:  1.2258213 Prediction:  f you want you
2675 loss:  1.2257656 Prediction:  f you want you
2676 loss:  1.2257097 Prediction:  f you want you
2677 loss:  1.2256539 Prediction:  f you want you
2678 loss:  1.2255982 Prediction:  f you want you
2679 loss:  1.2255425 Prediction:  f you want you
2680 loss:  1.2254871 Prediction:  f you want you
268

2883 loss:  1.215169 Prediction:  f you want you
2884 loss:  1.2151226 Prediction:  f you want you
2885 loss:  1.2150761 Prediction:  f you want you
2886 loss:  1.2150296 Prediction:  f you want you
2887 loss:  1.2149833 Prediction:  f you want you
2888 loss:  1.214937 Prediction:  f you want you
2889 loss:  1.2148907 Prediction:  f you want you
2890 loss:  1.2148445 Prediction:  f you want you
2891 loss:  1.2147982 Prediction:  f you want you
2892 loss:  1.2147521 Prediction:  f you want you
2893 loss:  1.214706 Prediction:  f you want you
2894 loss:  1.2146599 Prediction:  f you want you
2895 loss:  1.2146138 Prediction:  f you want you
2896 loss:  1.2145678 Prediction:  f you want you
2897 loss:  1.2145219 Prediction:  f you want you
2898 loss:  1.214476 Prediction:  f you want you
2899 loss:  1.2144299 Prediction:  f you want you
2900 loss:  1.2143843 Prediction:  f you want you
2901 loss:  1.2143383 Prediction:  f you want you
2902 loss:  1.2142925 Prediction:  f you want you
2903

### Really long sentence?

In [13]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [21]:
char_set = list(set(sentence))
char_dic = {w:i for i,w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
seq_length = 10
learning_rate = 0.1

dataX = []  #index값들을 넣는다
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i+seq_length]
    y_str = sentence[i+1:i+1+seq_length]
    print(i, x_str,'->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [41]:
tf.reset_default_graph()

X = tf.placeholder(tf.int32, [None, seq_length])
Y = tf.placeholder(tf.int32, [None, seq_length])

x_one_hot = tf.one_hot(X, num_classes)

cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size, state_is_tuple = True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state = initial_state, dtype = tf.float32)

weights = tf.ones([batch_size, seq_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = Y, weights = weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)

prediction = tf.argmax(outputs, axis = 2)

긴 문장은 결과가 좋지 않음
- 너무 깊이가 얕기 때문

In [43]:
with tf.Session(config = config) as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(500):
        l, _ = sess.run([loss, train], feed_dict = {X:dataX, Y:dataY})
        results = sess.run(prediction, feed_dict = {X:dataX})
        
        if i % 10 == 0:
            print(results)
            for j, result in enumerate(results):
                if j == 0:
                    print(i, ''.join([char_set[c] for c in result]), end = '')
                else:
                    print(char_set[result[-1]] , end = '')
            print('\n')

[[ 1  2 24 ... 14 14 14]
 [17 24 21 ... 14 14 14]
 [24 22 14 ... 14 14 14]
 ...
 [22 24 14 ... 13 13 13]
 [24 14 17 ... 13 13 13]
 [19  7 24 ... 13 13 13]]
0 yftwaaaaaaaaaaaap.  gatttnwwwwbbo a   yyyaaag,abugniynndndgnddeabbrgryyyillaaaalaoo  iwwcywwwwnnnnnnppwwwwwaaabeepppppwpdnpgpnngrtnrnnnnnnndoollldwpdnnngn nnht nnnn,c,nwadwwdnnnnn

[[ 1  2 24 ... 14 14 14]
 [17 24 21 ... 14 14 14]
 [24 22 14 ... 14 14 14]
 ...
 [22 24 14 ... 13 13 13]
 [24 14 24 ... 13 13 13]
 [19  7 24 ... 13 13 13]]
10 yftwaaaaaaaaaaaapa  eatttnwwwwbbo a   oo aawgaabueniynndndgnddeabbrgryyyillaaaalao   iwhcyww whnnnnnppwwwwwaaabeepppppwpdnpgnnngrtnrnnnnnn doollldw dnnnnn nnht nnnn,c,nwadwwdnnnnn

[[ 1  2 24 ... 14 14 14]
 [17 24 21 ... 14 14 14]
 [24 22 14 ... 14 14 14]
 ...
 [22 24 14 ... 13 13 13]
 [24 14 24 ... 13 13 13]
 [19  7 24 ... 13 13 13]]
20 yftwaaaaaaaa aaapa  eattthwwwwbo      oo aawgaabuenilnndndgndteabbrgryyyillaaoalao   i  chww whnnnnnp wwwwwaaabeepppppwpdnpg nngrtnrnnnnnn ooollldw dnnn n nn t nn

260                                                                                                                                                                                    

[[ 8  8  8 ...  8  8  8]
 [ 8 24  8 ...  8  8  8]
 [24  8  8 ...  8  8  8]
 ...
 [ 8  8  8 ...  8  8  8]
 [24  8  8 ...  8  8  8]
 [15  8 24 ...  8  8  8]]
270                                                                                                                                                                                    

[[ 8  8  8 ...  8  8  8]
 [ 8 24  8 ...  8  8  8]
 [24  8  8 ...  8  8  8]
 ...
 [ 8  8  8 ...  8  8  8]
 [24  8  8 ...  8  8  8]
 [15  8  8 ...  8  8  8]]
280                                                                                                                                                                                    

[[ 8  8  8 ...  8  8  8]
 [ 8 24  8 ...  8  8  8]
 [24  8  8 ...  8  8  8]
 ...
 [ 8  8  8 ...  8  8  8]
 [24  8  8 ...  8  8  8]
 [1